In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import glob

import pandas as pd

In [9]:
big_data = pd.DataFrame()
files = glob.iglob('backtests/binance/' + '**/backtest_result.txt', recursive=True)
i = 0
for filename in files:
    #print(f'parsing : {filename}')
    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': 'C:\\AgodaGit\\passivbot\\' + filename}
        current_state = 'long'
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Passivbot Version':
                    d['v'] = value
                if key == 'Symbol':
                    d['s'] = value

                if key == 'Long':
                    current_state = 'long'
                    d['long'] = bool(value) or False
                if key == 'Short':
                    current_state = 'short'
                    d['short'] = bool(value) or False

                if key == 'Total gain' and current_state == 'long':
                    d['tgl_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'long':
                    d['adgl_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'long':
                    d['bnkrl_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'long':
                    d['ebrl'] = float(value.replace('%', '').strip())

                if key == 'Total gain' and current_state == 'short':
                    d['tgs_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'short':
                    d['adgs_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'short':
                    d['bnkrs_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'short':
                    d['ebrs'] = float(value.replace('%', '').strip())

        big_data = big_data.append(d, ignore_index=True)

big_data

C:\Users\skhan\AppData\Local\Temp/ipykernel_271948/3439014496.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_271948/3439014496.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_271948/3439014496.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_271948/3439014496.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_

,file,v,s,ebrl,long,short,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrs,tgl_%,tgs_%
0,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.909573,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.114573,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.838780,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.952475,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.242079,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,C:\AgodaGit\passivbot\backtests/binance\XTZUSD...,5.3,XTZUSDT,0.972171,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
505,C:\AgodaGit\passivbot\backtests/binance\XTZUSD...,5.3,XTZUSDT,0.847562,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
506,C:\AgodaGit\passivbot\backtests/binance\XTZUSD...,5.3,XTZUSDT,0.841806,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
507,C:\AgodaGit\passivbot\backtests/binance\ZECUSD...,5.4,ZECUSDT,0.966600,True,True,0.06,0.0179,100.0,100.0,0.9792,12.53,3.573


Now filter the dataframe

In [12]:
allowed_symbols = ["ETHUSDT", "XRPUSDT", "1000SHIBUSDT", "DOGEUSDT", "ADAUSDT",
                   "BNBUSDT", "MATICUSDT", "DOTUSDT", "SANDUSDT", "FTMUSDT",
                   "LTCUSDT", "LINKUSDT", "MANAUSDT", "ETCUSDT", "EOSUSDT",
                   "ATOMUSDT", "FILUSDT", "ICPUSDT", "ALICEUSDT", "LRCUSDT",
                   "BCHUSDT", "SUSHIUSDT", "XTZUSDT", "THETAUSDT", "CRVUSDT",
                   "ALGOUSDT", "CHZUSDT", "TRXUSDT", "VETUSDT", "CHRUSDT",
                   "ONEUSDT", "OMGUSDT", "XLMUSDT", "RUNEUSDT", "ENJUSDT",
                   "CELRUSDT", "SXPUSDT", "1INCHUSDT", "SRMUSDT", "COTIUSDT",
                   "ZECUSDT", "QTUMUSDT", "GRTUSDT", "DENTUSDT", "SFPUSDT",
                   "STORJUSDT", "IOTAUSDT", "HOTUSDT", "NEOUSDT", "COMPUSDT",
                   "LINAUSDT", "KAVAUSDT", "BATUSDT", "WAVESUSDT", "IOSTUSDT",
                   "ALPHAUSDT", "RLCUSDT", "HBARUSDT", "DODOUSDT", "XMRUSDT",
                   "SNXUSDT", "ANKRUSDT", "DASHUSDT", "REEFUSDT", "ZENUSDT",
                   "CVCUSDT", "RENUSDT", "ICXUSDT", "RSRUSDT", "SKLUSDT",
                   "NKNUSDT", "BELUSDT", "ONTUSDT", "OGNUSDT", "BLZUSDT",
                   "MTLUSDT", "CTKUSDT", "RVNUSDT", "LITUSDT", "MKRUSDT",
                   "OCEANUSDT", "ZILUSDT", "UNFIUSDT", "BANDUSDT", "YFIIUSDT",
                   "TOMOUSDT", "TRBUSDT", "KNCUSDT", "STMXUSDT", "ZRXUSDT",
                   "SCUSDT", "AKROUSDT", "XEMUSDT", "FLMUSDT", "BALUSDT",
                   "BTSUSDT", "DGBUSDT", "DEFIUSDT"]

allowed_symbols = ['FTMUSDT']

df = big_data[(big_data.s.isin(allowed_symbols))
              & (big_data['long'] == True)
              & (big_data['short'] == True)

              #& (big_data['bnkrl_%'] > 95.0)
              #& (big_data['bnkrs_%'] > 95.0)

              & (big_data['v'] == '5.4')

              # & (big_data['tg_%'] > 10)
              # & (big_data['ebr'] > .85)
              ]
df

,file,v,s,ebrl,long,short,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrs,tgl_%,tgs_%
272,C:\AgodaGit\passivbot\backtests/binance\FTMUSD...,5.4,FTMUSDT,0.9493,True,True,0.0661,0.393,100.0,9.724,0.09727,13.92,0.0450
275,C:\AgodaGit\passivbot\backtests/binance\FTMUSD...,5.4,FTMUSDT,0.9493,True,True,0.0661,0.393,100.0,9.724,0.09727,13.92,0.0450
277,C:\AgodaGit\passivbot\backtests/binance\FTMUSD...,5.4,FTMUSDT,0.9399,True,True,0.1060,0.392,100.0,9.745,0.09745,23.54,0.0714
278,C:\AgodaGit\passivbot\backtests/binance\FTMUSD...,5.4,FTMUSDT,0.9404,True,True,0.1090,0.392,100.0,9.745,0.09745,24.20,0.0714
279,C:\AgodaGit\passivbot\backtests/binance\FTMUSD...,5.4,FTMUSDT,0.9446,True,True,0.1090,0.393,100.0,9.718,0.09722,24.33,0.0450


In [11]:
df.drop_duplicates(subset=['s', 'tgl_%', 'tgs_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long', 'short'], keep='first',
                   inplace=True)
df

,file,v,s,ebrl,long,short,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrs,tgl_%,tgs_%
